In [ ]:
from selenium.webdriver.support.ui import WebdriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from pandas.io import sql
import os
import pandas as pd
# from decouple import config
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from PIL import Image
from matplotlib import cm
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

from selenium import webdriver
from tqdm import tqdm
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");

browser = webdriver.Chrome('./chromedriver', options=options)

In [ ]:
# enter the channel
browser.get("https://www.youtube.com/channel/UCFa0g-8D8hYuHvdIh6XQCOg")

# click the video tab
browser.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()

# scroll down
body = brwoser.find_element_by_css_selector('body')
for i in range(32):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [3]:
youtube_dict = {"title":[], "viewCount":[],
                "uploadDate":[], "likes":[],
                "hates":[], "info":[],
                "tags":[], "vidLength":[],
                "commentsCount":[], "comment":[]}

# total video count: 596
for i in tqdm(range(1, 596)):
    browser.find_element_by_xpath(f'/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[{str(i)}]/div[1]/div[1]/div[1]/h3/a').click()
    
    time.sleep(10)
    
    browser.find_element_by_xpath('//*[@id="movie_player"]/div[1]/video').click()
    
    # vid title
    name = browser.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
    youtube_dict["title"].append(name)
    time.sleep(1)
    
    # viewCount
    watch = browser.find_element_by_xpath('').text
    watch = int("".join(re.findall("[0-9]{1,}",watch)))
    youtube_dict["viewCount"].append(watch)
    time.sleep(1)
    
    # uploadDate
    day = browser.find_element_by_xpath('').text
    day = day.replace(" ","")[:-1]
    youtube_dict["uploadDate"].append(day)
    time.sleep(1)
    
    # page_source
    # To find the likes & hates
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser') ## 여기 머임?
    
    # likes
    try:
        good = html.find('yt-formatted-string', {'id': 'text', 'class': 'style-scope ytd-toggle-button-renderer style-text',
                                                'aria-label': re.compile('좋아요')}).text
        good = int(good)
        youtube_dict['likes'].append(good)
    except:
        youtube_dict['likes'].append("")
    time.sleep(1)
    
    # hates
    try:
        hate = html.find('yt-formatted-string', {'id': 'text', 'class': 'style-scope ytd-toggle-button-renderer style-text',
                                                'aria-label': re.compile('싫어요')}).text
        hate = int(hate)
        youtube_dict['hates'].append("")
    time.sleep(1)
    except:
        youtube_dict['hates'].append("")
    time.sleep(1)
    
    # info
    outline_test = html.find("yt-formatted-string", {"class": "content style-scope ytd-video-secondary-info-renderer"}).text
    # add try&except for no tag vids
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        youtube_dict["tags"].append(hash_tag)
    except Exception as e: # 여기도 왜?
        youtube_dict["tags"].append("")
        
    if hash_tag:
        try:
            outline = outline_test.replace(hash_tag, "")
            youtube_dict['info'].append(outline)
        except:
            youtube_dict['info'].append("")
    else:
        try:
            youtube_dict['info'].append(outline_test)
        except:
            youtube_dict['info'].append("")
    time.sleep(1)
    
    # video length - 왜 html로 찾는지?
    video_length = html.find("span", {"class":"ytp-time-duration"}).text
    youtube_dict["vidLength"].append(video_length)
    time.sleep(1)
    
    # scroll for vertical-friendly vids
    body = browser.find_element_by_css_selector('body')
    for i in range(3):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    html0 = browser.page_source
    html = BeautifulSoup(html0, 'html.parser')
    
    # commentsCount
    try:
        comment_num = html.find('yt-formatted-string', {"class":"count-text style-scope ytd-comments-header-renderer"}).text
        comment_num = int(re.findall("[0-9]{1,}", comment_num)[0])
        youtube_dict['commentsCount'].append(comment_num)
    except: 
        youtube_dict["commentsCount"].append("")
    time.sleep(1)
    
    # comment(list)
    comment_lst = []
    # 
    try:
        comments = html.findall('yt-formatted-string', {'id':'content-text', 'class': 'style-scope ytd-comment-renderer'})
        for i in comments: 
            comment = i.text
            comment_lst.append(comment)
    except:
        comment_lst.append("")
    
    comment_lst = "\z".join(comment_lst) # 질문?
    youtube_dict['comment'].append(comment_lst)
    
    browser.get('https://www.youtube.com/channel/UCFa0g-8D8hYuHvdIh6XQCOg/videos')

SyntaxError: invalid syntax (<ipython-input-3-32c94642c12a>, line 53)

In [ ]:
pd.DataFrame(youtube_dict)